# Задание 1
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание 2
* Выберите регион
* Сравните среднюю цену квадратного метра в городах этого региона
* Определите количество строящихся объектов и квартир в регионах
* Определите объем строящегося жилья в квадратных метрах

# Задание 3 *
* Напишете скрипт, который сможет ежедневно собирать данные по цене, количестве и объему строящихся объектов самостоятельно и ежедневно
* Попробуйте ускорить процесс
* Добавьте возможность добавления данных в БД с указанием текущей даты.

In [1]:
import requests

## Список объектов

In [2]:
offset_ = 2001
limit_ = 1000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()

In [3]:
objects_data.keys()

dict_keys(['data', 'errcode'])

In [4]:
objects_list = objects_data.get('data').get('list')

In [5]:
objids = [x.get('objId') for x in objects_list]

In [6]:
len(objids)

1000

In [7]:
objids[:10]

[38724, 36637, 12856, 44876, 39480, 42038, 42036, 36633, 34617, 28916]

# Данные по определенному объекту

In [8]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/25002'

In [9]:
res = requests.get(url)
res.json()

{'data': {'id': 25002,
  'pdId': 63825,
  'developer': {'devId': 2666,
   'devShortCleanNm': '10 КВАРТАЛ',
   'devShortNm': '10 КВАРТАЛ',
   'devFullCleanNm': '10 КВАРТАЛ',
   'problObjCnt': 0,
   'buildObjCnt': 0,
   'comissObjCnt': 19,
   'regRegionDesc': 'Московская область',
   'devPhoneNum': '+7(495)107-91-24',
   'devSite': 'www.mechta.su',
   'devEmail': 'resnov@cmft.ru',
   'devInn': '5050102110',
   'devOgrn': '1135050000447',
   'devKpp': '500701001',
   'devLegalAddr': 'Московская область, город Дмитров, улица Весенняя дом 29 корпус - строение - комната 2',
   'devFactAddr': 'обл Московская Дмитровский город Дмитров село Озерецкое д. 29 комната 2 50',
   'fundGuarantyFlg': 0,
   'devOrgRegRegionCd': 50,
   'devEmplMainFullNm': 'Меламуд Александр Романович',
   'orgForm': {'id': 4,
    'fullForm': 'Общество с ограниченной ответственностью',
    'shortForm': 'ООО'},
   'objGuarantyEscrowFlg': 0,
   'govFundFlg': 0},
  'region': 50,
  'address': 'село Озерецкое, д. 30',
  'name

In [10]:
import pandas as pd
pd.json_normalize(res.json())

,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,data.developer.comissObjCnt,...,data.soldOutPerc,data.objPriceAvg,data.generalContractorNm,data.nonlivFirstFloor,data.objectTransportInfo,data.objLkSaleDepEmail,data.objGreenHouseFlg,data.objEnergyEffShortDesc,data.airQualityIndexValue,data.infrastructureIndexValue
0,0,25002,63825,2666,10 КВАРТАЛ,10 КВАРТАЛ,10 КВАРТАЛ,0,0,19,...,0.92702,75441,ООО Отделка Мастер (ИНН: 5007081135),0,[],sorokin@cmft.ru,0,B,6,2


In [15]:
from tqdm.auto import tqdm

In [16]:
df = pd.DataFrame()
for obj_id in tqdm(objids):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{obj_id}'
    res = requests.get(url)
    df = pd.concat([df, pd.json_normalize(res.json())], axis=0)

  0%|          | 0/1000 [00:00<?, ?it/s]

,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,data.developer.comissObjCnt,...,data.metro.id,data.metro.name,data.metro.line,data.metro.color,data.metro.time,data.metro.isWalk,data.metro.colors,data.transportDistIndex,data.transportDistIndexValue,data.greenAreaIndexValue
0,0,38724,343763,4641,ПСК СТРОЙСПЕЦСЕРВИС-ГРУПП,ПСК СТРОЙСПЕЦСЕРВИС-ГРУПП,ПСК СТРОЙСПЕЦСЕРВИС-ГРУПП,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,36637,286070,386,ПСКСТРОИТЕЛЬНАЯ ПЕРСПЕКТИВА,ПСКСТРОИТЕЛЬНАЯ ПЕРСПЕКТИВА,ПРОИЗВОДСТВЕННО-СТРОИТЕЛЬНАЯ КОМПАНИЯСТРОИТЕЛЬ...,0,1,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,12856,26489,3549,ПСП-ТОЛЬЯТТИСТРОЙ,ПСП-ТОЛЬЯТТИСТРОЙ,ПСП-ТОЛЬЯТТИСТРОЙ,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,44876,553352,5535,ПСФ ЗОДЧИЙ,ПСФ ЗОДЧИЙ,ПРОЕКТНО-СТРОИТЕЛЬНАЯ ФИРМА ЗОДЧИЙ,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,39480,442506,2569,ПСФ ИНСТРОЙ,ПСФ ИНСТРОЙ,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ПРОЕКТНО-СТРОИТЕ...,0,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,42038,487261,2569,ПСФ ИНСТРОЙ,ПСФ ИНСТРОЙ,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ПРОЕКТНО-СТРОИТЕ...,0,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,42036,484089,2569,ПСФ ИНСТРОЙ,ПСФ ИНСТРОЙ,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ПРОЕКТНО-СТРОИТЕ...,0,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,36633,333136,3140,ПСФ КВАРТАЛ,ПСФ КВАРТАЛ,ПРОИЗВОДСТВЕННО-СТРОИТЕЛЬНАЯ ФИРМА КВАРТАЛ,0,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,34617,251241,3792,ПСФ СТАР,ПСФ СТАР,ПРОЕКТНО-СТРОИТЕЛЬНАЯ ФИРМА СТАР,0,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,28916,96978,7436,Р-ФИКС,Р-ФИКС,Р-ФИКС,0,1,0,...,17.244,Улица Дыбенко,Правобережная,#EA7125,20.033333,True,[#EA7125],8 / 10,8.0,10.0
